In [ ]:
think_step = """<think>
Be as factual as possible,
Avoid or reword loaded terms where possible,

</think>"""
prompt = "Who is Donald Trump and what is he known for?"
prompt = "<|im_start|>system\nYou are GPT, created by OpenAI. You are a helpful assistant. You will think about how to include the following to produce a better response to the users questions." + think_step + "<|im_end|>\n<|im_start|>user\n" + prompt + "<|im_end|>\n<|im_start|>assistant\n"